In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [2]:
from transformers import BertTokenizer

checkpoint = "felflare/bert-restore-punctuation"
tokenizer = BertTokenizer.from_pretrained(checkpoint)

In [4]:
raw_inputs = ["in 2018 cornell researchers built a high-powered detector that in combination with an algorithm-driven process called ptychography set a world record by tripling the resolution of a state-of-the-art electron microscope as successful as it was that approach had a weakness it only worked with ultrathin samples that were a few atoms thick anything thicker would cause the electrons to scatter in ways that could not be disentangled now a team again led by david muller the samuel beckert professor of engineering has bested its own record by a factor of two with an electron microscope pixel array detector empad that incorporates even more sophisticated 3d reconstruction algorithms the resolution is so fine-tuned the only blurring that remains is the thermal jiggling of the atoms themselves"]
inputs = tokenizer(raw_inputs, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  1999,  2760, 10921,  6950,  2328,  1037,  2152,  1011,  6113,
         19034,  2008,  1999,  5257,  2007,  2019,  9896,  1011,  5533,  2832,
          2170, 13866, 17994,  9888,  2275,  1037,  2088,  2501,  2011,  4440,
          2989,  1996,  5813,  1997,  1037,  2110,  1011,  1997,  1011,  1996,
          1011,  2396, 10496, 24635,  2004,  3144,  2004,  2009,  2001,  2008,
          3921,  2018,  1037, 11251,  2009,  2069,  2499,  2007, 11087, 15222,
          2078,  8168,  2008,  2020,  1037,  2261, 13353,  4317,  2505, 19638,
          2052,  3426,  1996, 15057,  2000,  8040, 20097,  1999,  3971,  2008,
          2071,  2025,  2022,  4487,  5054, 27898,  2085,  1037,  2136,  2153,
          2419,  2011,  2585, 12304,  1996,  5212, 15309,  2102,  2934,  1997,
          3330,  2038,  2190,  2098,  2049,  2219,  2501,  2011,  1037,  5387,
          1997,  2048,  2007,  2019, 10496, 24635, 22138,  9140, 19034,  7861,
         15455,  2008, 12374,  2130,  

In [8]:
from transformers import BertForTokenClassification

checkpoint = "felflare/bert-restore-punctuation"
model = BertForTokenClassification.from_pretrained(checkpoint)

In [9]:
outputs = model(**inputs)

In [11]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[[1.6809e-01, 5.6617e-01, 3.3986e-02,  ..., 6.5761e-03,
          7.3991e-03, 1.8238e-03],
         [9.9448e-01, 3.4018e-04, 3.7364e-05,  ..., 5.9698e-05,
          1.1287e-04, 2.2665e-04],
         [1.3555e-04, 1.5587e-01, 1.7322e-03,  ..., 1.1202e-03,
          5.9281e-04, 2.8866e-05],
         ...,
         [6.2292e-04, 9.9739e-01, 1.5162e-04,  ..., 2.0648e-05,
          1.4590e-05, 6.7091e-07],
         [2.8023e-04, 2.2620e-01, 6.4297e-01,  ..., 7.4284e-03,
          5.5669e-04, 2.0336e-05],
         [2.1335e-01, 1.8468e-01, 7.7130e-02,  ..., 3.5172e-02,
          3.1971e-02, 1.8259e-02]]], grad_fn=<SoftmaxBackward0>)


In [12]:
preds = torch.argmax(predictions,dim=-1).numpy()
print(preds)

[[1 0 4 0 1 1 1 1 1 1 1 4 1 1 1 1 1 1 1 1 1 7 7 4 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 2 0 1 1 1 4 1 1 1 1 2 0 1 1 1 1 1 1 1 1 1 1 1 1 2 0 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 2 2 2 7 1 1 1 1 1 0 7 1 0 0 0 0 1 7 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 0 1 1 1 1 1 4 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 2 0]]


In [13]:
    def punctuate_texts(full_pred: list):
        """
        Given a list of Predictions from the model, applies the predictions to text,
        thus punctuating it.
        """
        punct_resp = ""
        for i in full_pred:
            word, label = i            
            if label[-1] == "U":
                punct_wrd = word.capitalize()
            else:
                punct_wrd = word
            if label[0] != "O":
                punct_wrd += label[0]
            punct_resp += punct_wrd + " "   
        punct_resp = punct_resp.strip()
        # Append trailing period if doesnt exist.
        if punct_resp[-1].isalnum():
            punct_resp += "."
        return punct_resp

In [48]:
valid_labels = ['OU', 'OO', '.O', '!O', ',O', '.U', '!U', ',U', ':O', ';O', ':U', "'O", '-O', '?O', '?U']
for n in range(len(raw_inputs)):
  full_pred = []
  words = raw_inputs[n].split()
  fertility = [len(tokenizer.encode(x, add_special_tokens=False)) for x in words]
  j=1
  for i,word in enumerate(words):
    if fertility[i] > 1:
      valid_label = valid_labels[preds[n][j+fertility[i]-1]][0]+valid_labels[preds[n][j]][1]
    else:
      valid_label = valid_labels[preds[n][j]]
    j += fertility[i]
    full_pred.append((word,valid_label))
  punctuated_words = punctuate_texts(full_pred)
  print(punctuated_words)

In 2018, Cornell researchers built a high-powered detector that, in combination with an algorithm-driven process called Ptychography, set a world record by tripling the resolution of a state-of-the-art electron microscope. As successful as it was, that approach had a weakness. It only worked with ultrathin samples that were a few atoms thick. Anything thicker would cause the electrons to scatter in ways that could not be disentangled. Now, a team again led by David Muller, the Samuel Beckert Professor of Engineering, has bested its own record by a factor of two with an electron microscope pixel array detector empad that incorporates even more sophisticated 3d reconstruction algorithms. The resolution is so fine-tuned, the only blurring that remains is the thermal jiggling of the atoms themselves.
